# 基于通义千问原生接口的ReAct代理实现
https://python.langchain.com/docs/use_cases/tool_use/prompting/

In [1]:
#导入语言模型
import os
from langchain_community.llms import Tongyi
from langchain_community.llms import SparkLLM
from langchain_community.llms import QianfanLLMEndpoint

import pandas as pd
#导入模版
from langchain.prompts import PromptTemplate

#导入聊天模型
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)

from langchain_community.chat_models import ChatSparkLLM
from langchain_community.chat_models.tongyi import ChatTongyi
from langchain_community.chat_models import QianfanChatEndpoint

#输入三个模型各自的key

os.environ["DASHSCOPE_API_KEY"] = ""

os.environ["IFLYTEK_SPARK_APP_ID"] = ""
os.environ["IFLYTEK_SPARK_API_KEY"] = ""
os.environ["IFLYTEK_SPARK_API_SECRET"] = ""

os.environ["QIANFAN_AK"] = ""
os.environ["QIANFAN_SK"] = ""

from operator import itemgetter
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

In [2]:
model_ty = Tongyi(temperature=0)
model_qf = QianfanLLMEndpoint(model="ERNIE-Bot")
chat_qf = QianfanChatEndpoint(model="ERNIE-Bot")
chat_xh = ChatSparkLLM()
chat_ty= ChatTongyi()

In [3]:
import langchain
langchain.__version__

'0.1.9'

In [7]:
#! pip install --upgrade langchain -i https://mirrors.aliyun.com/pypi/simple
#! pip install --upgrade langchain_community -i https://mirrors.aliyun.com/pypi/simple

## 通义千问原生接口调用

https://help.aliyun.com/zh/dashscope/developer-reference/api-details?spm=a2c4g.11186623.0.0.65eb12b0M4nTKZ

https://blog.csdn.net/qq_53280175/article/details/134746680

In [3]:
from dashscope import Generation
import dashscope
from langchain.tools.render import render_text_description
 
dashscope.api_key = ""

In [4]:
def get_response(mess):
    response = Generation.call(
        model='qwen-turbo',
        messages=mess,
        result_format='message', # 将输出设置为message形式
    )
    return response

In [5]:
res=get_response([{"content":"hi","role":"user"}])#user system assistant
res

GenerationResponse(status_code=<HTTPStatus.OK: 200>, request_id='57ea59d2-db83-9d9e-9a85-7f697f1b02ba', code='', message='', output=GenerationOutput(text=None, choices=[Choice(finish_reason='stop', message=Message({'role': 'assistant', 'content': 'Hello! How can I assist you today?'}))], finish_reason=None), usage=GenerationUsage(input_tokens=20, output_tokens=9))

In [6]:
res.output.choices[0].message["content"]

'Hello! How can I assist you today?'

### 模版的使用

In [7]:
query="量子力学有什么用"

physics_template = f"""你是一个非常聪明的物理学教授。
你擅长以简洁易懂的方式回答物理问题。
当你不知道答案时，你会承认不知道。

这是一个问题：
{query}"""

print(physics_template)

你是一个非常聪明的物理学教授。
你擅长以简洁易懂的方式回答物理问题。
当你不知道答案时，你会承认不知道。

这是一个问题：
量子力学有什么用


In [8]:
res=get_response([{"content":physics_template,"role":"user"}])

In [10]:
print(res.output.choices[0].message["content"])

量子力学是一门基础且极其重要的物理学分支，它描述了微观世界的行为，如原子、分子、光子和更小粒子的运动。它的应用广泛且深远，尽管在日常生活中我们可能觉察不到，但对现代科技发展有着重大影响：

1. **技术基础**：量子力学是现代信息技术的基石，包括量子计算、量子通信（如量子密钥分发）、以及量子密码学（如量子随机数生成）等前沿领域。

2. **材料科学**：量子效应在新型材料如超导体、半导体、量子点和量子阱中起着关键作用，这些材料在电子设备（如电脑芯片）和太阳能电池等方面有重要应用。

3. **医学成像**：核磁共振成像（MRI）就利用了量子力学中的核自旋现象。

4. **能源**：量子力学原理在理解光合作用和太阳能电池的工作原理中不可或缺。

5. **纳米技术**：在制造和操控极小尺度的物体时，量子力学的规则变得至关重要。

6. **量子纠缠**：这是一种神奇的现象，两个粒子即使相隔遥远也能瞬间相互影响，这为量子通信提供了可能性。

7. **基础研究**：量子力学本身也是物理学的基石，推动了我们对宇宙本质的理解，比如黑洞信息悖论和宇宙起源的研究。

总的来说，量子力学不仅是理论研究的前沿，也是推动科技进步和社会发展的重要力量。


### 与langchain工具的结合使用

In [11]:
from langchain_core.tools import tool

@tool
def multiply(first_int: int, second_int: int) -> int:
    """将两个整数相乘。"""
    return first_int * second_int

@tool
def add(first_int: int, second_int: int) -> int:
    "将两个整数相加。"
    return first_int + second_int

@tool
def exponentiate(base: int, exponent: int) -> int:
    "对底数求指数幂。"
    return base**exponent

In [12]:
from langchain_core.utils.function_calling import convert_to_openai_tool

In [14]:
convert_to_openai_tool(multiply)

{'type': 'function',
 'function': {'name': 'multiply',
  'description': 'multiply(first_int: int, second_int: int) -> int - 将两个整数相乘。',
  'parameters': {'type': 'object',
   'properties': {'first_int': {'type': 'integer'},
    'second_int': {'type': 'integer'}},
   'required': ['first_int', 'second_int']}}}

In [13]:
def get_response_1t(mess):
    response = Generation.call(
        model='qwen-turbo',
        messages=mess,
        tools=[convert_to_openai_tool(multiply)],
        result_format='message', # 将输出设置为message形式
    )
    return response

In [16]:
res=get_response_1t([{"content":"5加9","role":"user"}])

In [17]:
res.output.choices[0]

Choice(finish_reason='stop', message=Message({'role': 'assistant', 'content': '5加9等于14。这是一个基本的算术加法。如果你需要进一步的帮助或者有更复杂的数学问题，随时告诉我。'}))

### classwork 1

* 通过通义千问使用一个工具
* 一下是用LECL实现的一个多工具代理，请通过通义千问原生的工具接口实现多工具的择优

In [20]:
#[convert_to_openai_tool(i) for i in tools]

In [18]:
tools = [add, exponentiate, multiply]

def tool_chain(model_output):
    tool_map = {tool.name: tool for tool in tools}
    chosen_tool = tool_map[model_output["name"]]
    return itemgetter("arguments") | chosen_tool

In [21]:
def get_response_3t(mess):
    response = Generation.call(
        model='qwen-turbo',
        messages=mess,
        tools=[convert_to_openai_tool(i) for i in tools],
        result_format='message', # 将输出设置为message形式
    )
    return response

In [22]:
res=get_response_3t([{"content":"5加9","role":"user"}])

In [29]:
res_agrs=res.output.choices[0].message['tool_calls'][0]["function"]

In [35]:
res_agrs["arguments"]=parser.parse(res_agrs["arguments"])

In [36]:
res_agrs

{'name': 'add', 'arguments': {'first_int': 5, 'second_int': 9}}

In [37]:
t_run=tool_chain(res_agrs)
t_run.invoke(res_agrs)

14

In [24]:
from langchain.tools.render import render_text_description

rendered_tools = render_text_description(tools)
system_prompt = f"""您是一名助理，可以使用以下工具集。 以下是每个工具的名称和说明:

{rendered_tools}

根据用户输入，返回要使用的工具的名称和输入。 将您的响应作为带有'name'和'arguments'键的 JSON blob 返回，“arguments”键对应的值应该是所选函数的输入参数的字典，字典里不要有任何说明,此JSON blob必须是如下格式：```json
...
```"""

prompt = ChatPromptTemplate.from_messages(
    [("system", system_prompt), ("user", "{input}")]
)

In [25]:
chain = prompt | model_ty | JsonOutputParser() | tool_chain
chain.invoke({"input": "3加上1234"})

1237

In [32]:
from langchain_core.utils.function_calling import convert_to_openai_tool
from langchain_core.output_parsers import JsonOutputParser

#tools_p=[convert_to_openai_tool(i) for i in tools]
parser = JsonOutputParser()
output_dict = parser.parse('{"result": "2 + 2 = 4", "mode": "math_operation"}')
output_dict

{'result': '2 + 2 = 4', 'mode': 'math_operation'}

## 基于ReAct提示词框架的代理实现

### ReAct提示词框架

原始论文：

https://arxiv.org/pdf/2210.03629.pdf

论文解读

https://zhuanlan.zhihu.com/p/660951271


langchain的实现方案解读

https://cloud.tencent.com/developer/article/2286923

https://cloud.tencent.com/developer/article/2267317

langchain AgentExecutor方法的源代码：

https://api.python.langchain.com/en/latest/_modules/langchain/agents/agent.html#AgentExecutor

langchain 相关prompt 代码的地址：

https://github.com/langchain-ai/langchain/blob/master/libs/langchain/langchain/agents/mrkl/prompt.py

### 基于通义千问的react代理实现

* classwork 2

1. 通义千问导入上面三个工具

2. 借用langchain agent模版，构建模版函数，完成模型的初始调用

3. 结合初次调用结果，形成下次调用的prompt，并完成二次调用

4. 再次结合完成最终的调用

5. 综合上面所有代码，构建循环完成一个ReAct代理的实现

In [ ]:
PREFIX = """Answer the following questions as best you can. You have access to the following tools:"""
FORMAT_INSTRUCTIONS = """Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question"""
SUFFIX = """Begin!

Question: {input}
Thought:{agent_scratchpad}"""

In [73]:
tools = [add, exponentiate, multiply]
parser = JsonOutputParser()
output_dict = parser.parse('{"result": "2 + 2 = 4", "mode": "math_operation"}')
tool_names=[i.name for i in tools]


def prompt_react(input):
    PREFIX = """Answer the following questions as best you can. You have access to the following tools:"""
    p0= f"""Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take in order, should be one of {tool_names}
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:"""
    return [{"role":"system","content":PREFIX},{"role":"user","content":p0}]

def tool_chain(model_output):
    tool_map = {tool.name: tool for tool in tools}
    chosen_tool = tool_map[model_output["name"]]
    return itemgetter("arguments") | chosen_tool

def get_response_3t(mess):
    response = Generation.call(
        model='qwen-turbo',
        messages=mess,
        tools=[convert_to_openai_tool(i) for i in tools],
        result_format='message', # 将输出设置为message形式
    )
    return response
def get_args(res_c):
    args0=res_c.split("\nAction: ")[1].split("\nAction Input:")
    return {"name":args0[0],"arguments":parser.parse(args0[1])}

In [85]:
p_r=prompt_react("4加5乘以6然后再算4次方然后再加35")

for i in range(0,8):
    res=get_response_3t(p_r)
    res_content=res.output.choices[0].message["content"]
    print(res_content)
    if res_content.find("Action Input:")!=-1:
        args=get_args(res_content)
        t_run=tool_chain(args)
        p_r[1]["content"]=p_r[1]["content"]+res_content+"\nObservation: "+str(t_run.invoke(args))+"\n"
    else:
        p_r[1]["content"]=p_r[1]["content"]+res_content
        break
print(p_r[1]["content"])

 我们需要按照数学中的运算顺序来计算这个问题，先乘法后加法，最后是幂运算。所以步骤应该是：1. 5乘以6，2. 将结果加上4，3. 计算得到的和的四次方，4. 最后加上35。我会使用相应的工具来进行计算。
Action: multiply
Action Input: {"first_int": 5, "second_int": 6}
Thought: 现在我们得到了乘法的结果，下一步是将这个结果加上4。
Action: add
Action Input: {"first_int": 30, "second_int": 4}
Thought: 加上4后的结果是34，接下来我们需要对这个34进行四次方运算。
Action: exponentiate
Action Input: {"base": 34, "exponent": 4}
Thought: 现在已经得到了四次方的结果，最后一步是把这个值加上35。
Action: add
Action Input: {"first_int": 1336336, "second_int": 35}
Thought: 最终答案是1336371。
Final Answer: 1336371
Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take in order, should be one of ['add', 'exponentiate', 'multiply']
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: 4加5乘以

### 完整代码

 我们需要先计算5和6的乘积，然后加上4，最后将这个结果立方。这涉及到两个数学步骤：乘法和幂运算。我会先做乘法，然后做幂运算。
Action: multiply
Action Input: {"first_int": 5, "second_int": 6}
现在我得到了5和6的乘积，下一步是将其与4相加，然后计算这个和的三次方。
Action: add
Action Input: {"first_int": 30, "second_int": 4}
我已经得到了34这个和，接下来我需要对34进行三次方运算。
Action: exponentiate
Action Input: {"base": 34, "exponent": 3}
我已经计算出了最终的答案，5乘以6再加4的结果的三次方等于39,304。
Final Answer: 39,304
Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take in order, should be one of ['add', 'exponentiate', 'multiply']
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: 5乘以6加上4然后再算其3次方
Thought: 我们需要先计算5和6的乘积，然后加上4，最后将这个结果立方。这涉及到两个数学步骤：乘法和幂运算。我会先做乘法，然后做幂运算。
Action: multiply
Action Input: {"first_int": 5, "second_in

## 百度千帆原生接口的使用

https://cloud.baidu.com/doc/WENXINWORKSHOP/s/xlmokikxe#%E5%A4%9A%E8%BD%AE%E5%AF%B9%E8%AF%9D

百度的特点：

* 原生也支持function call
* 默认会出发搜索工具，配合搜索工具产生答案

In [29]:
import os
import qianfan

# 使用安全认证AK/SK鉴权，通过环境变量初始化；替换下列示例中参数，安全认证Access Key替换your_iam_ak，Secret Key替换your_iam_sk
os.environ["QIANFAN_ACCESS_KEY"] = ""
os.environ["QIANFAN_SECRET_KEY"] = ""

chat_comp = qianfan.ChatCompletion()

# 指定特定模型
resp = chat_comp.do(model="ERNIE-Bot", messages=[{
    "role": "user",
    "content": "你好"
}], enable_citation=True)

print(resp)


[INFO] [04-23 09:37:55] openapi_requestor.py:316 [t:8396]: requesting llm api endpoint: /chat/completions
[INFO] [04-23 09:37:55] oauth.py:207 [t:8396]: trying to refresh access_token for ak `og6mWr***`
[INFO] [04-23 09:37:55] oauth.py:220 [t:8396]: sucessfully refresh access_token


QfResponse(code=200, headers={'Access-Control-Allow-Headers': 'Content-Type', 'Access-Control-Allow-Origin': '*', 'Appid': '53521775', 'Connection': 'keep-alive', 'Content-Encoding': 'gzip', 'Content-Type': 'application/json; charset=utf-8', 'Date': 'Tue, 23 Apr 2024 01:37:56 GMT', 'P3p': 'CP=" OTI DSP COR IVA OUR IND COM "', 'Server': 'Apache', 'Set-Cookie': 'BAIDUID=B165DBCFBE0BDB333783DFE61A98BC53:FG=1; expires=Thu, 31-Dec-37 23:55:55 GMT; max-age=2145916555; path=/; domain=.baidu.com; version=1', 'Statement': 'AI-generated', 'Vary': 'Accept-Encoding', 'X-Aipe-Self-Def': 'eb_total_tokens:24,prompt_tokens:1,id:as-z4wt2mr6i8', 'X-Baidu-Request-Id': 'sdk-py-0.3.2-9Je862bKWA1V6IZ4', 'X-Openapi-Server-Timestamp': '1713836275', 'X-Ratelimit-Limit-Requests': '300', 'X-Ratelimit-Limit-Tokens': '300000', 'X-Ratelimit-Remaining-Requests': '299', 'X-Ratelimit-Remaining-Tokens': '299999', 'Content-Length': '349'}, body={'id': 'as-z4wt2mr6i8', 'object': 'chat.completion', 'created': 1713836276, 

### 千帆自带的搜索工具触发

In [30]:
def wenda_qf(mes):
    resp = chat_comp.do(model="ERNIE-Bot", messages=[{
        "role": "user",
        "content": mes
    }], enable_citation=True)

    return resp

res=wenda_qf("杭州师范大学最近有什么新闻")

[INFO] [04-23 09:38:01] openapi_requestor.py:316 [t:8396]: requesting llm api endpoint: /chat/completions


In [31]:
res.body

{'id': 'as-r9x52wwhxs',
 'object': 'chat.completion',
 'created': 1713836290,
 'result': '杭州师范大学最近的新闻有^[2]^：\n\n* **杭州师范大学主办2024年全国污染生态学学术研讨会** 。该研讨会于4月14日至16日在杭州召开，会议主题为"污染生态与环境健康"。\n* **杭州师范大学本科教育教学审核评估线上评估启动会召开** 。该会议于4月15日下午召开，旨在全面贯彻落实省市"新春第一会"精神，坚持"四个面向"办教育，始终胸怀"国之大者"。\n\n此外，杭州师范大学还有召开第四届教职工代表大会等新闻^[1]^。',
 'is_truncated': False,
 'need_clear_history': False,
 'search_info': {'is_beset': 0,
  'rewrite_query': '杭州师范大学最近有什么新闻',
  'search_results': [{'index': 1,
    'url': 'https://www.hznu.edu.cn/test/xw/',
    'title': '杭州师范大学新闻网',
    'datasource_id': 'aurora-1'},
   {'index': 2,
    'url': 'https://www.hznu.edu.cn/index.shtml?eqid=ff9fff0a0004a2d600000004646f041c',
    'title': '杭州师范大学',
    'datasource_id': 'aurora-1'}]},
 'finish_reason': 'normal',
 'usage': {'prompt_tokens': 5, 'completion_tokens': 118, 'total_tokens': 123}}

In [32]:
res.body["search_info"]#保存搜索结果的位置

{'is_beset': 0,
 'rewrite_query': '杭州师范大学最近有什么新闻',
 'search_results': [{'index': 1,
   'url': 'https://www.hznu.edu.cn/test/xw/',
   'title': '杭州师范大学新闻网',
   'datasource_id': 'aurora-1'},
  {'index': 2,
   'url': 'https://www.hznu.edu.cn/index.shtml?eqid=ff9fff0a0004a2d600000004646f041c',
   'title': '杭州师范大学',
   'datasource_id': 'aurora-1'}]}

### 百度千帆原生接口function call的使用

https://www.cnblogs.com/szj666/p/17938812

In [ ]:
import qianfan
import re
from pprint import pprint
import json
prompt = "想去北京天安门附近的10公里内的旅游景点" 

def attractionRecommend(arguments):
    return "天坛公园"
def attractionReservation(arguments):
    return "预约成功"
def function_calling(prompt):
    print("user-prompt:"+prompt)
    model = "ERNIE-Bot"
    print('=' * 30,'大模型-',model,' 输出 ', '='*30,"\n")
    response = chat_comp.do(
            model=model, 
            messages=[{
                "role": "user",
                "content": prompt
                }],
            temperature=0.000000001,
            functions=[
                {
                    "name": "attractionRecommend",
                    "description": "景点推荐",
                    "parameters": {
                        "type": "object",
                        "properties": {
                            "location": {
                                "type": "string",
                                "description": "地址信息，包括省市、街道、门牌号等"
                                },
                            "expect_distance": {
                                "type": "int",
                                "description": "距离"
                                }
                            },
                        "required": ["location"]
                        },
                    "responses": {
                        "type": "object",
                        "properties": {
                            "price": {
                                "type": "int",
                                "description": "景点距离"
                                },
                            "food": {
                                "type": "string",
                                "description": "景点名称"
                                },
                            },
                        },
                 },
                 {
                        "name": "attractionReservation",
                        "description": "景点介绍预约",
                        "parameters": {
                            "type": "object",
                            "properties": {
                                
                                "attraction": {
                                    "type": "string",
                                    "description": "景点名称"
                                    },
                                },
                            "required": ["attraction"]
                            },
                        "responses": {
                            "type": "object",
                            "properties": {
                                "result": {
                                    "type": "string",
                                    "description": "回答完成"
                                    },
                                }
                            },
                   }
                 ]
            )


    pprint(response)
    return response
chat_comp = qianfan.ChatCompletion()


prompt_list = re.split(r"----", prompt)

for prompt in prompt_list:
    response = function_calling(prompt)
    function_name = response['body']['function_call']['name']
    arguments = response['body']['function_call']['arguments']
    print(eval(function_name)(arguments))
    
    #拿到response后，解析json，调用自定义的数据表api和下单api
    print("\n")
    print('=' * 30,"大模型响应结束","="*30)
